# Cargamos librerias y conectamos la base de datos

In [63]:
from pymongo import MongoClient
import time, json, random
from random import randrange
from datetime import timedelta, datetime
from random import randint
from lorem_text import lorem
from flask import jsonify

client = MongoClient('localhost', 27017)

In [64]:
db = client["test_mongo"]

# Generación de datos aleatorios

In [4]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [5]:

d1 = datetime.strptime('1/1/1950 12:59 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2022 01:00 AM', '%m/%d/%Y %I:%M %p')
print(random_date(d1, d2))  

2021-11-15 08:11:06


In [6]:
descripciones = [
    'Transferencia interbancaria SIMPLE',
    'DEB.CTA.P/C.INTERNET',
    'Debito Cta por ACH',
    'Retiro en efectivo',
    'AB.CTA.P/MOVIL',
    'Pago en interés',
    'Debito Cta por compra',
    'Debito Cta por pago de Seguro',
    'Retención IVA'
]

# Creación de índices

In [7]:
db.extracto_cuenta.create_index('fecha')
db.extracto_cuenta.create_index('agencia')
db.extracto_cuenta.create_index('monto')
db.extracto_cuenta.create_index('saldo')

'saldo_1'

In [8]:
db.extracto_cuenta.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'fecha_1': {'v': 2, 'key': [('fecha', 1)]},
 'agencia_1': {'v': 2, 'key': [('agencia', 1)]},
 'monto_1': {'v': 2, 'key': [('monto', 1)]},
 'saldo_1': {'v': 2, 'key': [('saldo', 1)]}}

# Aproximadamente 100 000 registros

## Carga de datos

### De forma individual

In [14]:
start_time = time.time()

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }
    
    db.extracto_cuenta.insert_one(insert_data)


end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:01:06.528760


### De forma masiva

In [16]:
insert_data_list = []

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    insert_data_list.append(insert_data)
start_time = time.time()
db.extracto_cuenta.insert_many(insert_data_list)
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:03.057983


In [47]:
insert_data_list = []

for i in range():
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    insert_data_list.append(insert_data)
start_time = time.time()
db.extracto_cuenta.insert_many(insert_data_list)
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:08.671126


### Un solo dato

In [17]:
start_time = time.time()

fecha = random_date(d1, d2)
agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
monto = round(random.uniform(10.0, 4000.0), 2)
descripcion = random.choice(descripciones)
saldo = round(random.uniform(0.0, 10000000.0), 2)
nota = lorem.words(randint(1,13))

insert_data = {
    'fecha': fecha,
    'agencia': agencia,
    'monto': monto,
    'descripcion': descripcion,
    'saldo': saldo,
    'nota': nota
}

db.extracto_cuenta.insert_one(insert_data)


end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:00.006199


In [18]:
db.extracto_cuenta.count_documents({})

100001

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [19]:
def busqueda_fecha_agencia(month, year, agencia):
    from_date_str = f"{year}-{month}-01" + "T00:00:00+00:00"
    to_date_str = f"{year}-{month}-30" + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
    criteria = {"$and": [{"fecha": {"$gte": from_date, "$lte": to_date}}, {"agencia": f"{agencia}"}]}

    return db.extracto_cuenta.find(criteria)

In [30]:
start_time = time.time()
results = busqueda_fecha_agencia(month=5, year = 2021, agencia = "La Paz")
end_time = time.time()

for result in results[:10]:
    print(result)


print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Se han encontrado {results.count()} documentos")


{'_id': ObjectId('63635237dfb1bb15465c92b4'), 'fecha': datetime.datetime(2021, 5, 3, 19, 55, 41), 'agencia': 'La Paz', 'monto': 1842.29, 'descripcion': 'Debito Cta por compra', 'saldo': 8519692.88, 'nota': 'nesciunt totam deserunt maxime aspernatur excepturi voluptas ducimus soluta'}
{'_id': ObjectId('636351c0dfb1bb15465b434a'), 'fecha': datetime.datetime(2021, 5, 3, 23, 17, 22), 'agencia': 'La Paz', 'monto': 1292.5, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 8367632.87, 'nota': 'magnam amet harum nisi similique voluptatibus quidem enim alias fugiat'}
{'_id': ObjectId('636351dadfb1bb15465b8b41'), 'fecha': datetime.datetime(2021, 5, 4, 5, 58, 22), 'agencia': 'La Paz', 'monto': 1304.89, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 4303362.51, 'nota': 'nostrum'}
{'_id': ObjectId('636351ccdfb1bb15465b63e5'), 'fecha': datetime.datetime(2021, 5, 4, 17, 51, 20), 'agencia': 'La Paz', 'monto': 668.93, 'descripcion': 'Debito Cta por compra', 'saldo': 7382384.59, 'nota': 'be

/tmp/ipykernel_5446/2146191820.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


### Búsqueda por montos mayores a determinado número y agencia

In [32]:
def busqueda_monto_mayor(monto, agencia):
    criteria = {"$and": [{"monto": {"$gte": monto}}, {"agencia": agencia}]}

    return db.extracto_cuenta.find(criteria)

In [33]:
start_time = time.time()
results =  busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Se han encontrado {results.count()} documentos")

{'_id': ObjectId('636351badfb1bb15465b367e'), 'fecha': datetime.datetime(1987, 3, 16, 17, 21, 10), 'agencia': 'Cochabamba', 'monto': 3831.34, 'descripcion': 'Debito Cta por ACH', 'saldo': 8656977.68, 'nota': 'qui dolore mollitia iure consequuntur aperiam sint illum inventore laboriosam temporibus quaerat minus'}
{'_id': ObjectId('636351badfb1bb15465b368c'), 'fecha': datetime.datetime(1955, 5, 4, 1, 41, 28), 'agencia': 'Cochabamba', 'monto': 3561.57, 'descripcion': 'Debito Cta por compra', 'saldo': 9202183.25, 'nota': 'necessitatibus assumenda commodi ex nobis reprehenderit veniam natus'}
{'_id': ObjectId('636351badfb1bb15465b36e7'), 'fecha': datetime.datetime(2016, 5, 2, 10, 42, 49), 'agencia': 'Cochabamba', 'monto': 3773.91, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7211470.13, 'nota': 'dolorem est impedit'}
{'_id': ObjectId('636351badfb1bb15465b370c'), 'fecha': datetime.datetime(2013, 5, 5, 9, 38, 47), 'agencia': 'Cochabamba', 'monto': 3148.24, 'descripcion': 'AB.CTA.P/MOVIL', 

/tmp/ipykernel_5446/592893741.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


### Búsqueda considerando varios campos

In [34]:
def busqueda_compleja(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    criteria = {
        "$and": [
            {"$or": [{"agencia": agencia1}, {"agencia": agencia2}]},
            {"saldo": {"$gte": saldo_min, "$lte": saldo_max}},
            {"descripcion": descripcion}
        ]}

    return db.extracto_cuenta.find(criteria)

In [35]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=100000000, agencia1="Cochabamba", agencia2="Santa Cruz", descripcion="DEB.CTA.P/C.INTERNET")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print(f"Se han encontrado {results.count()} documentos")

{'_id': ObjectId('636351badfb1bb15465b36e7'), 'fecha': datetime.datetime(2016, 5, 2, 10, 42, 49), 'agencia': 'Cochabamba', 'monto': 3773.91, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 7211470.13, 'nota': 'dolorem est impedit'}
{'_id': ObjectId('636351badfb1bb15465b36e8'), 'fecha': datetime.datetime(1966, 2, 23, 0, 19, 23), 'agencia': 'Cochabamba', 'monto': 229.2, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 2923374.87, 'nota': 'aspernatur aliquid recusandae vel veniam itaque velit'}
{'_id': ObjectId('636351bbdfb1bb15465b376b'), 'fecha': datetime.datetime(1976, 11, 5, 6, 43, 36), 'agencia': 'Cochabamba', 'monto': 131.07, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 6848426.77, 'nota': 'odit reiciendis eligendi veritatis quo harum alias dolorem quasi autem'}
{'_id': ObjectId('636351bbdfb1bb15465b3773'), 'fecha': datetime.datetime(1960, 1, 27, 8, 6, 39), 'agencia': 'Cochabamba', 'monto': 2648.89, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 5485217.69, 'nota': 'laborum exerc

/tmp/ipykernel_5446/1390024898.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


### Agregaciones: Monto promedio por agencia

In [36]:
def promedio_monto_agencia(agencia):
    criteria = [
        {
        "$group" : 
            {"_id" : {"agencia" : f"{agencia}"},  
            "Monto promedio" : {"$avg" : "$monto"}
            }
        }
     ]

    return db.extracto_cuenta.aggregate(criteria)

In [37]:
start_time = time.time()
results =  promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

for i in results:
    print(i)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'_id': {'agencia': 'La Paz'}, 'Monto promedio': 2006.700515794842}
En un tiempo de 0:00:01.003458


## Actualización de datos

In [38]:
def actualizacion_fecha_monto(min_date, max_date, monto):
    from_date_str = min_date + "T00:00:00+00:00"
    to_date_str = max_date + "T23:59:59+00:00"
    from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
    to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
    
    criteria = {"$and": [{"fecha": {"$gte": from_date, "$lte": to_date}}]}
    new = {"$set": { "monto" : monto, "nota": "Monto actualizado por fecha"}}

    return db.extracto_cuenta.update_many(criteria, new)

In [39]:
from_date_str = "2021-01-01" + "T00:00:00+00:00"
to_date_str = "2021-01-31" + "T23:59:59+00:00"
from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
db.extracto_cuenta.count_documents({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

112

In [40]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2021-01-01", max_date = "2021-01-31", monto = 7000)
end_time = time.time()


print(f"Se han actualizado {results.modified_count} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se han actualizado 112 documentos
En un tiempo de 0:00:00.169765


In [41]:
results = db.extracto_cuenta.find({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

for result in results[:10]:
    print(result)

{'_id': ObjectId('63635237dfb1bb15465c0b04'), 'fecha': datetime.datetime(2021, 1, 1, 12, 23, 34), 'agencia': 'Beni', 'monto': 7000, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 5632689.64, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63635237dfb1bb15465c27de'), 'fecha': datetime.datetime(2021, 1, 1, 15, 6, 18), 'agencia': 'Chuquisaca', 'monto': 7000, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 1216823.21, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636351dbdfb1bb15465b8f91'), 'fecha': datetime.datetime(2021, 1, 1, 19, 24, 11), 'agencia': 'La Paz', 'monto': 7000, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 4942227.13, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636351e0dfb1bb15465b9b9c'), 'fecha': datetime.datetime(2021, 1, 2, 0, 54, 27), 'agencia': 'Santa Cruz', 'monto': 7000, 'descripcion': 'Retiro en efectivo', 'saldo': 44012.01, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636351d1dfb1bb15465b7545'), 'fecha': datetime.d

## Eliminar datos

In [65]:
def eliminar_saldo_min(saldo_min):
    return db.extracto_cuenta.delete_many(
        {"saldo": {"$lte": saldo_min}}
)

In [43]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 500)
end_time = time.time()

print(f"Se han eliminado {results.deleted_count} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se han eliminado 5 documentos
En un tiempo de 0:00:00.002011


In [44]:
results = db.extracto_cuenta.count_documents({"saldo": {"$lte": 500}})
print(results)

0


# Más de 10 millones de documentos

## Carga de datos

In [45]:
insert_data_list = []

for _ in range(10000000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }

    insert_data_list.append(insert_data)

start_time = time.time()
db.extracto_cuenta.insert_many(insert_data_list)
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:19:59.770422


In [48]:
db.extracto_cuenta.count_documents({})

10099000

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [51]:
start_time = time.time()
results = busqueda_fecha_agencia(month=5, year = 2021, agencia = "La Paz")
end_time = time.time()

for result in results[:10]:
    print(result)
    

print("="*300)
print(f"Se han encontrado {results.count()} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")


{'_id': ObjectId('636355f4dfb1bb1546980f92'), 'fecha': datetime.datetime(2021, 5, 1, 0, 22, 58), 'agencia': 'La Paz', 'monto': 3695.91, 'descripcion': 'Retiro en efectivo', 'saldo': 681841.44, 'nota': 'itaque ad maiores culpa'}
{'_id': ObjectId('636355e6dfb1bb15468a68ba'), 'fecha': datetime.datetime(2021, 5, 1, 0, 28, 29), 'agencia': 'La Paz', 'monto': 2710.07, 'descripcion': 'Pago en interés', 'saldo': 418319.45, 'nota': 'repudiandae impedit quae mollitia iusto tempora quidem deserunt eveniet facilis molestiae'}
{'_id': ObjectId('63635603dfb1bb1546beae33'), 'fecha': datetime.datetime(2021, 5, 1, 1, 7, 43), 'agencia': 'La Paz', 'monto': 3839.84, 'descripcion': 'Pago en interés', 'saldo': 776690.25, 'nota': 'consectetur corrupti doloribus rem numquam tempora dolores necessitatibus dolorum quae animi'}
{'_id': ObjectId('636355b4dfb1bb15465f5b0a'), 'fecha': datetime.datetime(2021, 5, 1, 2, 21, 30), 'agencia': 'La Paz', 'monto': 454.73, 'descripcion': 'Debito Cta por pago de Seguro', 'sald

/tmp/ipykernel_5446/2112155487.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


### Búsqueda por montos mayores a determinado número y agencia

In [50]:
start_time = time.time()
results =  busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"Se han encontrado {results.count()} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")


{'_id': ObjectId('636355a5dfb1bb15465cbd25'), 'fecha': datetime.datetime(2006, 12, 7, 2, 6, 29), 'agencia': 'Cochabamba', 'monto': 3736.94, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 2799906.89, 'nota': 'deleniti fuga et consequuntur voluptates quas dolorum nam cupiditate incidunt enim accusantium ipsam'}
{'_id': ObjectId('636355a5dfb1bb15465cbd35'), 'fecha': datetime.datetime(1956, 6, 19, 4, 36, 12), 'agencia': 'Cochabamba', 'monto': 3841.46, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 807030.67, 'nota': 'explicabo earum non sunt unde vero ducimus aliquid deleniti recusandae perferendis'}
{'_id': ObjectId('636355a5dfb1bb15465cbd4d'), 'fecha': datetime.datetime(1991, 7, 4, 15, 58, 24), 'agencia': 'Cochabamba', 'monto': 3727.81, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 3130378.49, 'nota': 'ab repellat libero unde enim nulla delectus'}
{'_id': ObjectId('636355a5dfb1bb15465cbd79'), 'fecha': datetime.datetime(2010, 6, 14, 4, 47, 51), 'agencia': 'Cochabamba', 'monto': 3345.68, 'descripc

/tmp/ipykernel_5446/2480498654.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


Se han encontrado 316144 documentos
En un tiempo de 0:00:00.000159


### Búsqueda considerando varios campos

In [52]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=100000000, agencia1="Cochabamba", agencia2="Santa Cruz", descripcion="DEB.CTA.P/C.INTERNET")
end_time = time.time()

for result in results[:10]:
    print(result)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"Se han encontrado {results.count()} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'_id': ObjectId('636355a5dfb1bb15465cbd35'), 'fecha': datetime.datetime(1956, 6, 19, 4, 36, 12), 'agencia': 'Cochabamba', 'monto': 3841.46, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 807030.67, 'nota': 'explicabo earum non sunt unde vero ducimus aliquid deleniti recusandae perferendis'}
{'_id': ObjectId('636355a5dfb1bb15465cbd46'), 'fecha': datetime.datetime(1999, 1, 17, 22, 12, 33), 'agencia': 'Cochabamba', 'monto': 444.83, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 2472175.31, 'nota': 'nulla'}
{'_id': ObjectId('636355a5dfb1bb15465cbd73'), 'fecha': datetime.datetime(1994, 5, 24, 8, 9, 29), 'agencia': 'Cochabamba', 'monto': 29.82, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 878935.55, 'nota': 'omnis reiciendis ducimus sit ab quas facilis est totam ipsam'}
{'_id': ObjectId('636355a5dfb1bb15465cbd8e'), 'fecha': datetime.datetime(2012, 10, 15, 5, 58, 54), 'agencia': 'Cochabamba', 'monto': 3253.93, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 9583474.52, 'nota': 'voluptat

/tmp/ipykernel_5446/3361313736.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f"Se han encontrado {results.count()} documentos")


Se han encontrado 280963 documentos
En un tiempo de 0:00:00.000458


### Agregaciones: Monto promedio por agencia

In [53]:
start_time = time.time()
results =  promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

for i in results:
    print(i)
    
#print(f"Se encontraron {results.count()} registros")
print("="*300)
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'_id': {'agencia': 'La Paz'}, 'Monto promedio': 2005.9227867145262}
En un tiempo de 0:00:16.932983


## Actualización de datos

In [58]:
from_date_str = "2021-03-01" + "T00:00:00+00:00"
to_date_str = "2021-03-31" + "T23:59:59+00:00"
from_date = datetime.strptime(from_date_str, '%Y-%m-%dT%H:%M:%S%z')
to_date = datetime.strptime(to_date_str, '%Y-%m-%dT%H:%M:%S%z')
db.extracto_cuenta.count_documents({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

11947

In [59]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2021-03-01", max_date = "2021-03-31", monto = 7000)
end_time = time.time()


print(f"Se han actualizado {results.modified_count} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se han actualizado 11947 documentos
En un tiempo de 0:00:01.492238


In [60]:
results = db.extracto_cuenta.find({"$and": [{"fecha": {"$gte": from_date}}, {"fecha": {"$lte": to_date}}]})

for result in results[:10]:
    print(result)

{'_id': ObjectId('636355fadfb1bb1546a78aef'), 'fecha': datetime.datetime(2021, 3, 1, 0, 3, 6), 'agencia': 'Beni', 'monto': 7000, 'descripcion': 'Pago en interés', 'saldo': 8384659.59, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63635601dfb1bb1546b85792'), 'fecha': datetime.datetime(2021, 3, 1, 0, 26, 7), 'agencia': 'Santa Cruz', 'monto': 7000, 'descripcion': 'Retención IVA', 'saldo': 2909794.25, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636355f5dfb1bb15469a173f'), 'fecha': datetime.datetime(2021, 3, 1, 0, 26, 9), 'agencia': 'Beni', 'monto': 7000, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 6916964.54, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('63635610dfb1bb1546d79be3'), 'fecha': datetime.datetime(2021, 3, 1, 0, 26, 20), 'agencia': 'Beni', 'monto': 7000, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 8346770.05, 'nota': 'Monto actualizado por fecha'}
{'_id': ObjectId('636355b1dfb1bb15465e391c'), 'fecha': datetime.datetime(2021, 3, 1, 0, 34, 53), 'agen

## Eliminar datos

In [66]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 1000)
end_time = time.time()

print(f"Se han eliminado {results.deleted_count} documentos")
print(f"En un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se han eliminado 483 documentos
En un tiempo de 0:01:14.968753


In [62]:
results = db.extracto_cuenta.count_documents({"saldo": {"$lte": 1000}})
print(results)

0
